In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import  numpy as np 
import tensorflow as tf 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler


2025-08-18 22:11:25.561968: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755555085.574891       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755555085.579103       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 22:11:25.596301: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df_train = pd.read_csv("../input/tabular-playground-series-may-2022/train.csv")
df_train.head()


,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,0,-0.249088,0.530642,0.335227,0.806819,-0.184190,-0.560442,1.253767,2,1,...,-1.100706,-0.078688,2.160728,0.002502,-0.827445,ACBADABECB,158.820720,0,1,1
1,1,-0.312833,0.033082,-0.571193,1.311494,0.991718,-0.138249,1.834627,2,1,...,1.089416,-2.701227,-1.846792,9.539707,2.443596,BBBCAAAFDE,211.389880,0,0,0
2,2,-0.370032,1.501016,0.288983,0.077866,-0.329701,0.030314,0.384582,1,3,...,0.313065,0.293554,-3.490299,2.002782,0.593525,BDAEAABICD,81.745258,1,0,0
3,3,-1.011059,-0.341850,1.415175,-2.128660,1.137287,-3.200017,0.289945,7,2,...,-0.626560,0.989921,-0.760995,-3.276785,-2.534943,BAABFADDCA,-644.638548,0,0,1
4,4,0.756528,0.503700,0.935057,-2.321849,-0.002466,0.058981,-0.163493,1,0,...,-1.504338,0.632537,1.812423,2.883524,-1.089711,AABFBBEMHC,-140.205455,0,2,1


In [4]:
df_train.corr().tail(1)


ValueError: could not convert string to float: 'ACBADABECB'

In [5]:
f = plt.figure(figsize=(19, 15))
plt.matshow(df_train.corr(), fignum=f.number)
plt.xticks(range(df_train.select_dtypes(['number']).shape[1]), df_train.select_dtypes(['number']).columns, fontsize=14, rotation=45)
plt.yticks(range(df_train.select_dtypes(['number']).shape[1]), df_train.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);


ValueError: could not convert string to float: 'ACBADABECB'

<Figure size 1900x1500 with 0 Axes>

In [6]:
df_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 33 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      800000 non-null  int64  
 1   f_00    800000 non-null  float64
 2   f_01    800000 non-null  float64
 3   f_02    800000 non-null  float64
 4   f_03    800000 non-null  float64
 5   f_04    800000 non-null  float64
 6   f_05    800000 non-null  float64
 7   f_06    800000 non-null  float64
 8   f_07    800000 non-null  int64  
 9   f_08    800000 non-null  int64  
 10  f_09    800000 non-null  int64  
 11  f_10    800000 non-null  int64  
 12  f_11    800000 non-null  int64  
 13  f_12    800000 non-null  int64  
 14  f_13    800000 non-null  int64  
 15  f_14    800000 non-null  int64  
 16  f_15    800000 non-null  int64  
 17  f_16    800000 non-null  int64  
 18  f_17    800000 non-null  int64  
 19  f_18    800000 non-null  int64  
 20  f_19    800000 non-null  float64
 21  f_20    80

In [7]:
df_test = pd.read_csv("../input/tabular-playground-series-may-2022/test.csv")


In [8]:
for df in [df_train, df_test]:
    for i in range(10):
        df[f'ch{i}'] = df.f_27.str.get(i).apply(ord) - ord('A')
        
    df["unique_characters"] = df.f_27.apply(lambda s: len(set(s)))
    
features = [f for f in df_test.columns if f != 'id' and f != 'f_27']


In [9]:
X_train = df_train.drop(['target'],axis=1)[features]
Y_train = df_train['target'].to_numpy()
X_test = df_test[features].copy()


In [10]:
X_test.head()


,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,unique_characters
0,0.194491,1.180926,-1.474570,-1.969979,-0.632224,-0.908537,-0.929414,3,0,2,...,1,0,0,1,0,2,8,5,1,5
1,0.073512,1.215182,1.887183,0.664100,1.445916,0.407144,-0.965463,1,0,2,...,0,1,1,1,1,3,7,2,2,5
2,0.990606,1.390944,0.765208,-0.204159,-0.265682,0.051791,2.526925,6,4,5,...,4,1,1,1,0,0,11,2,1,5
3,0.621236,-1.282787,0.836890,1.208417,-0.212496,-1.729244,-0.741488,3,1,2,...,3,1,1,1,1,3,13,2,4,6
4,-0.143469,0.952492,-0.146993,0.362883,-0.105120,-1.208656,-1.317498,2,1,0,...,1,0,2,0,0,5,0,2,4,5


In [11]:
#Se puede realizar un normalizado, o ver las features y procesarlos para que tengan una distribución normal. 
StSc = StandardScaler()
X_train = StSc.fit_transform(X_train)
X_test  = StSc.transform(X_test)


In [12]:
print(X_train[0:5])
print(Y_train[0:5])


[[-2.48846255e-01  5.29914011e-01  3.33729930e-01  8.08681189e-01
  -1.83607086e-01 -5.61437628e-01  1.25463087e+00 -1.94325418e-02
  -6.65196569e-01 -8.32230776e-01 -1.32339732e+00  1.27574751e-01
   1.79146992e+00 -1.45603720e+00 -3.79092985e-01 -7.02081100e-01
   5.78959949e-01  9.64039453e-02 -6.80926789e-01 -3.52575480e-02
   3.81269211e-01  1.64488624e+00 -4.45586949e-01  1.18362406e-01
   1.04892112e+00 -7.24587265e-02 -4.78337737e-01  6.66879005e-01
  -7.26576260e-01 -3.50720641e-03 -9.98049402e-01  2.26939960e-02
   7.02895467e-01 -1.24582309e+00  1.24252939e+00 -9.36609855e-01
  -8.32364239e-01 -9.41120243e-01 -1.45236222e-02 -7.76949964e-01
  -1.59307278e-02]
 [-3.12646327e-01  3.18743858e-02 -5.72166042e-01  1.31348121e+00
   9.92180985e-01 -1.39249800e-01  1.83571766e+00 -1.94325418e-02
  -6.65196569e-01 -2.21586144e-01 -1.08143884e-01 -5.22765207e-01
  -1.04523161e+00 -1.55958280e-01  3.56276314e-01  1.21006937e+00
   5.78959949e-01 -1.26630433e+00  5.97411360e-01  7.4963

In [13]:
X_train.shape


(800000, 41)

In [14]:
X_test.shape


(100000, 41)

In [15]:
#activacion tanh dado que hay valores negativos, sino, normalizar con min max.
L2 = 0.000003
model_class = tf.keras.models.Sequential([
    tf.keras.layers.Input(41),
    tf.keras.layers.Dense(82, kernel_regularizer=tf.keras.regularizers.l2(L2),activation='swish'),
    tf.keras.layers.Dense(82, kernel_regularizer=tf.keras.regularizers.l2(L2),activation='swish'),
    tf.keras.layers.Dense(82, kernel_regularizer=tf.keras.regularizers.l2(L2),activation='swish'),
    tf.keras.layers.Dense(41, kernel_regularizer=tf.keras.regularizers.l2(L2),activation='swish'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])


ValueError: Cannot convert '41' to a shape.

In [16]:
#opt = tf.keras.optimizers.SGD(learning_rate=0.05, momentum=0.3, nesterov=False, name="SGD")
loss = tf.keras.losses.BinaryCrossentropy()
opt= tf.keras.optimizers.Adam()
model_class.compile(optimizer = opt,loss = loss,metrics = [tf.keras.metrics.BinaryAccuracy(),
                                                                               tf.keras.metrics.Precision(),
                                                                               tf.keras.metrics.Recall()])


2025-08-18 22:11:36.761865: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


NameError: name 'model_class' is not defined

In [17]:
earlystopping = tf.keras.callbacks.EarlyStopping( monitor= 'val_loss', patience=6, verbose=0,mode='auto', baseline=None, restore_best_weights=True)

LR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0,mode='auto')


In [18]:
history = model_class.fit(
    x=X_train,
    y=Y_train,
    batch_size=500,
    epochs=300,
    verbose=1,
    callbacks=[LR,earlystopping],
    validation_split=0.1,
    validation_data=None,
    shuffle=False,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=1800,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)


NameError: name 'model_class' is not defined

In [19]:
acc_train = history.history['binary_accuracy']
acc_val  =  history.history['val_binary_accuracy']

epochs = range(len(acc_train))
plt.plot(epochs, acc_train, 'r', label='Training accuracy')
plt.plot(epochs, acc_val, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()


NameError: name 'history' is not defined

In [20]:
pred = model_class.predict(X_test)
df_test['target'] = pred
submit   = df_test[['id','target']]
submit.to_csv("subsission.csv",index=False)


NameError: name 'model_class' is not defined